### **Vitamin D’s Impact on Autoimmune, Cardiovascular, and Cancer Diseases: Robustness Study**

This notebook evaluates the robustness of network-based proximity calculations between Vitamin D targets and disease-associated genes.

The objective is to assess how perturbations in the input data—such as different **Protein-Protein Interaction (PPI) networks**, disease-associated genes, and Vitamin D targets—impact the results obtained in the study **Exploring Vitamin D’s Impact on Autoimmune, Cardiovascular, and Cancer Diseases: A Network Medicine Perspective Case Study with NetMedPy**.

This robustness analysis complements the pipeline detailed in `MAIN_DIR/examples/VitaminD/VitD_pipeline.ipynb`.

#### **Prerequisites**
Before executing this notebook, ensure that the following preprocessing steps have been completed:

- **Generate additional PPI networks:** `examples/VitaminD/supplementary/sup_code/data_integration/BioNets.ipynb`
- **Extract Vitamin D targets:** `examples/VitaminD/supplementary/sup_code/data_integration/Vit_D_Targets.ipynb`

#### **Required Packages:**

`pandas, numpy, pickle, networkx, netmedpy, rtools, scipy upsetplot` 

#### **Workflow**
1. Load multiple PPI networks.
2. Compare network structures (nodes and edges).
3. Compute network-based proximity between Vitamin D and diseases using different PPI networks.
4. Evaluate the robustness of the results by analyzing rank correlations across different PPI networks.


### **Section 1: Changing the PPI network**

#### **Step 1: Loading required libraries**

In [ ]:
import pandas as pd
import numpy as np
import pickle
import networkx as nx
import netmedpy

import rtools as tools
from scipy.stats import spearmanr

import matplotlib.pyplot as plt
import seaborn as sns
from upsetplot import UpSet, from_memberships

# Number of cores used for calculation
n_procs = 30

### **Step 2: Load PPI Networks**

We load three different **Protein-Protein Interaction (PPI) networks** to evaluate the consistency of Vitamin D–disease proximity calculations:

- **Main PPI**: A manually curated, combined dataset integrating multiple sources.
- **BIOGRID PPI**: A human PPI network derived from the BIOGRID database, filtered by interaction confidence scores.
- **STRING PPI**: A human PPI network from STRING, filtered by `combined_score > 0.3` to ensure high-confidence interactions.

Each network is stored as a graph object using `networkx`, and only the **largest connected component (LCC)** is retained to ensure meaningful network proximity calculations.

In [ ]:
# Main PPI
main_ppi = tools.load("../../../data/input/ppi/ppi_network.pkl")

# BIOGRID PPI
biogrid_df = pd.read_csv("../../sup_data/alternative_ppi/ppi_biogrid.csv")
biogrid_df = biogrid_df.query("Score > 0")

biogrid_ppi = nx.from_pandas_edgelist(biogrid_df,source="Gene_Name_A",target="Gene_Name_B",create_using=nx.Graph())
biogrid_ppi = netmedpy.extract_lcc(biogrid_ppi.nodes, biogrid_ppi)

# STRING PPI
string_df = pd.read_csv("../../sup_data/alternative_ppi/ppi_string.csv")
string_df = string_df.query("combined_score > 0.3")

string_ppi = nx.from_pandas_edgelist(string_df, source="HGNC1", target = "HGNC2", create_using=nx.Graph())
string_ppi = netmedpy.extract_lcc(string_ppi.nodes, string_ppi)

ppi_dict = {
    "Main": main_ppi,
    "BIOGRID": biogrid_ppi,
    "STRING": string_ppi
}


In [ ]:
len(main_ppi.edges)

### **Step 3: Compare Network Size (Number of Nodes)**

Since different PPI networks are built from distinct data sources and filtering criteria, they may contain **different numbers of nodes**.

This step visualizes the node count for each network and their overlap, allowing us to assess differences in network structure before performing proximity calculations.

In [ ]:
node_to_sets = {}
for name, graph in ppi_dict.items():
    for node in graph.nodes:
        node_to_sets.setdefault(node, set()).add(name)

# Convert to membership format 
memberships = from_memberships(node_to_sets.values())

# Plot
plt.figure(figsize=(9, 5))
UpSet(memberships, subset_size='count', show_counts=True).plot()

for ax in plt.gcf().axes:
    for text in ax.texts:
        text.set_fontsize(8)  

plt.savefig("../../sup_plots/robustness/1_3_netsize_nodes.png", dpi=600, bbox_inches='tight')
plt.show()

### **Step 4: Compare Network Size (Number of Edges)**

We extend the comparison to include the **edges** in each network, which represents the number of protein-protein interactions.

A larger number of edges generally indicates a denser network, which may influence proximity calculations.

In [ ]:
# Helper function to normalize edges as sorted tuples
def get_edges_as_set(graph):
    return set(tuple(sorted(edge)) for edge in graph.edges)

# Get edge sets for each network
edge_sets = {
    name: get_edges_as_set(graph)
    for name, graph in ppi_dict.items()
}

# Create membership
edge_to_sets = {}
for name, edges in edge_sets.items():
    for edge in edges:
        edge_to_sets.setdefault(edge, set()).add(name)

# Convert to format for upsetplot
edge_memberships = from_memberships(edge_to_sets.values())

# Plot
plt.figure(figsize=(10, 5))
UpSet(edge_memberships, subset_size='count', show_counts=True).plot()

for ax in plt.gcf().axes:
    for text in ax.texts:
        text.set_fontsize(8)  

plt.savefig("../../sup_plots/robustness/1_4_netsize_edges.png", dpi=600, bbox_inches='tight')
plt.show()


### **Step 5: Load Vitamin D Targets and Disease Genes**

We load the following datasets:
- **Vitamin D targets**: A set of genes known to interact with or be influenced by Vitamin D.
- **Disease-associated genes**: Gene sets linked to various diseases of interest.

These datasets will be used in subsequent steps to compute network-based proximity scores.

In [ ]:
# Load drug targets
targets = tools.load("../../../data/input/drug_targets/vitd_targets_cpie.pkl")
vit_d = {"Vitamin D": targets}

# Load disease genes
disease_genes = tools.load("../../../data/input/disease_genes/disease_genes_merge.pkl")

### **Step 6: Compute Network Proximity for Each PPI Network**

For each PPI network, we calculate the **network proximity** between **Vitamin D targets** and **disease-associated genes**.

The proximity calculation involves:
1. Filtering Vitamin D targets and disease genes to ensure they exist in the given PPI.
2. Computing the **shortest path distance matrix** between all nodes in the network.
3. Using `netmedpy.screening` to calculate proximity scores based on shortest path distances.

The results are stored in `res_set`, where each key corresponds to a different PPI network.

In [ ]:
from time import sleep

# Collect the results in a dictionary
res_set = {}

for model,net in ppi_dict.items():
    print(model)
    vd = tools.filter_for_ppi(net,vit_d)
    dg = tools.filter_for_ppi(net,disease_genes) 

    ### Calculate shortest path distance matrix between node pairs
    sp_distance = netmedpy.all_pair_distances(net, 
                    distance="shortest_path", 
                    n_processors=n_procs, 
                    n_tasks=1000)

    ### CALCULATE PROXIMITY FROM VITAMIN D TO ALL DISEASES
    screen_data = netmedpy.screening(vd, 
                                    dg, 
                                    net, 
                                    sp_distance, 
                                    score="proximity", 
                                    properties=["z_score", "raw_amspl"],
                                    null_model="log_binning", 
                                    n_iter=10000, 
                                    n_procs=n_procs)
    res_set[model] = screen_data

    # Let netmedpy close properly.
    sleep(5)

### **Step 7: Visualize Proximity Scores**

We plot the proximity values of **Vitamin D** to different diseases across all PPI networks.

This visualization helps assess whether the results are **consistent across different networks** or whether certain diseases show significant variability.

In [ ]:
sns.set_theme(style="ticks", font_scale=1.0)

# Create a figure
plt.figure(figsize=(9, 6))

plt.axhline(y=0, color='gray', linestyle='--')

# Iterate over each PPI and plot the proximity
for name, data in res_set.items():
    df = data['z_score']
    diseases = df.columns
    values = df.loc["Vitamin D"].values
    sns.lineplot(x=diseases, y=values, marker='o', label=name)

plt.xlabel("")
plt.ylabel("Proximity to Vitamin D")
plt.xticks(rotation=45, ha="right")
plt.legend(loc='best')
plt.tight_layout()

plt.savefig("../../sup_plots/robustness/1_7_prox_vd.png", dpi=600, bbox_inches='tight')
plt.show()

### **Step 8: Evaluate Robustness Using Correlation Analysis**

To quantitatively assess the consistency of proximity scores across different PPI networks, we compute the **Spearman correlation** between proximity values from different networks.

Spearman correlation measures the similarity in ranking rather than absolute values, making it a robust metric for comparing proximity results.

In [ ]:
# Unify z-scores into a single DataFrame
z_score_data = {name: data['z_score'].loc["Vitamin D"] for name, data in res_set.items()}
z_score_df = pd.DataFrame(z_score_data)

# Calculate Spearman correlation
spearman_corr = z_score_df.corr(method='spearman')

sns.set_theme(style="ticks", font_scale=1.0)

# Plot heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(spearman_corr, annot=True, cmap="coolwarm", center=0, vmax=1, vmin=-1, linewidths=0.5, fmt=".2f")
plt.tight_layout()

# Show the plot
plt.savefig("../../sup_plots/robustness/1_8_correlation.png", dpi=600, bbox_inches='tight')
plt.show()

The correlation matrix demonstrates that the proximity results remain consistent across significantly different **Protein-Protein Interaction (PPI) networks**. This indicates that the relationship between **Vitamin D targets and diseases** is robust to variations in the underlying network structure.\n",


## **Section 2: Analyze the Effect of Target Reduction on Proximity Values**

To evaluate the stability of Vitamin D - Disease associations, we analyzed how proximity values change when the set of Vitamin D targets is randomly subsampled, simulating incomplete knowledge about drug-target interactions.


**How It Works:**
- We calculate the **mean proximity deviation** at each subsampling level, relative to the full dataset.
- A **small deviation** suggests that network proximity is robust to missing target genes.
- A **large deviation** indicates that proximity results are highly sensitive to the completeness of the target list.

The proximity deviation for each disease at a given subsampling level $p$ is calculated as:

$$
\Delta P_i(p) = P_i(p) - P_i(100\%)
$$

where:
- $P_i(p)$ is the proximity value for disease $i$ when using a **random subset** of $p\%$ of the original Vitamin D targets.
- $P_i(100\%)$ is the proximity value for disease $i$ using the **full set** of Vitamin D targets.
- $\Delta P_i(p)$ represents the change in proximity due to missing targets.

At each subsampling level, we take the **average deviation** across multiple runs to obtain a stable measure of the effect:

- If $\Delta P_i(p) \approx 0$ across all diseases, the proximity calculations are **robust**, meaning that even with missing targets, the proximity values remain stable.
- If $|\Delta P_i(p)|$ increases as $p$ decreases, it indicates **higher sensitivity** to missing target genes, suggesting that incomplete knowledge of drug targets **may significantly impact network-based proximity results**.

By plotting a **heatmap of $\Delta P_i(p)$** across all diseases and perturbation levels, we can visually assess which diseases are more or less affected by incomplete target information.


### **Step 1: Calculate Network Proximity Under Target subsampling**

We systematically sample the **Vitamin D target set** by selecting random subsets of decreasing size.

**Procedure:**
1. Calculate the baseline proximity using the **full** Vitamin D target set.
2. Iterate over different **subsampling levels** (90%, 80%, ..., 10% of targets retained).
3. For each level, randomly select a **subset of targets** and recalculate proximity.
4. Repeat multiple times at each level to account for random variability.
5. Store the proximity values for further analysis.

This allows us to evaluate the **robustness of proximity results** under uncertainty in target selection.

Be patient, we are doing a lot of computing here. It may take several hours.

In [ ]:

from time import sleep

# Define the perturbation levels and number of runs per level
net = ppi_dict['Main']
perturbation_levels = np.arange(10, 110, 10)  # From 10% to 100%
num_runs = 10  # Number of random subsets per level

# Dictionary to store proximity results for each perturbation level
proximity_results = {p: [] for p in perturbation_levels}

# Compute baseline proximity using the full set of Vitamin D genes
vd_full = tools.filter_for_ppi(net, vit_d)  # Full Vitamin D gene set
dg_filtered = tools.filter_for_ppi(net, disease_genes)  # Filtered disease gene sets
sp_distance = netmedpy.all_pair_distances(net, distance="shortest_path",
                                          n_processors=n_procs, n_tasks=1000)

screen_data_full = netmedpy.screening(vd_full, dg_filtered, net, sp_distance,
                                      score="proximity", properties=["z_score", "raw_amspl"],
                                      null_model="log_binning", n_iter=10000, n_procs=n_procs)

# Extract full-set proximity values
full_proximity = screen_data_full['z_score'].iloc[0]

sleep(5)

# Iterate over each perturbation level
for p in perturbation_levels:
    for i in range(num_runs):
        print(f"Level: {p} ; Run: {i}")
        # Randomly select a subset of Vitamin D genes
        subset_size = int(len(vd_full['Vitamin D']) * (p / 100))
        vd_subset = {'Vitamin D': np.random.choice(list(vd_full['Vitamin D']), subset_size, replace=False)}
        
        # Compute proximity with the subset
        screen_data_perturbed = netmedpy.screening(vd_subset, dg_filtered, net, sp_distance,
                                                   score="proximity", properties=["z_score", "raw_amspl"],
                                                   null_model="log_binning", n_iter=10000, n_procs=n_procs)
        
        # Store the proximity values
        proximity_results[p].append(screen_data_perturbed['z_score'].iloc[0])

        sleep(5)

# Convert results into a structured format
proximity_df = {p: np.vstack(proximity_results[p]) for p in perturbation_levels}

### **Step 2: Analyze the Effect of Target Reduction on Proximity Values**

To visualize the impact of reducing the number of Vitamin D targets, we calculate the **mean proximity deviation** at each subsampling level, relative to the full dataset. A **small deviation** suggests that network proximity is robust to missing target genes. A **large deviation** indicates that proximity results are highly sensitive to the completeness of the target list.

- If proximity remains stable across subsampling levels, it suggests that the method is **robust** to missing information.
- If deviations increase as more targets are removed, it suggests that **incomplete target information can significantly impact results**.

In [ ]:
# Compute mean deviation per disease and perturbation level
mean_deviation = {p: np.mean(proximity_df[p] - full_proximity.values, axis=0) for p in perturbation_levels}
df_heatmap = pd.DataFrame(mean_deviation, index=full_proximity.index)

# Plot heatmap of proximity deviations
plt.figure(figsize=(10, 6))
sns.heatmap(df_heatmap, cmap="coolwarm", annot=True, center=0, fmt=".2f")
plt.xlabel("Percentage of Vitamin D Targets Retained")
plt.ylabel("")

plt.savefig("../../sup_plots/robustness/2_2_deviation.png", dpi=600, bbox_inches='tight')
plt.show()

With the exception of **Inflammation**, reducing the target set by up to 20% (i.e., retaining 80% of targets) increases proximity values by less than one unit.

### **Step 3: Evaluate the Stability of Disease Ranking Under Vitamin D Targets Reduction**

Even if absolute proximity values change, it is important to assess whether the **relative ranking of diseases** remains consistent.

Here, we analyze the stability of disease ranking by:

1. Sorting diseases by proximity **(lower proximity = higher priority)**.
2. Calculating the **Spearman correlation** between proximity values using the full set of targets and rankings at different subsampling levels.
3. Plotting the correlation to determine how **consistent** the rankings remain under target reduction.

- A **high correlation across perturbation levels** indicates that disease prioritization remains **robust** even when removing Vitamin D targets.
- A **drop in correlation** suggests that disease rankings are highly **sensitive** to missing target genes.

This analysis provides insight into the reliability of network-based disease prioritization under real-world uncertainties.

In [ ]:
# Compute full-set disease ranking (lower proximity = higher priority)
full_rank = np.argsort(full_proximity.values) + 1

# Compute Spearman correlation for each run at each perturbation level
corr_distribution = {}

for p in perturbation_levels:
    correlations = []
    for run_proximity in proximity_df[p]:
        run_rank = np.argsort(run_proximity) + 1
        corr, _ = spearmanr(run_rank, full_rank)
        correlations.append(corr)
    corr_distribution[p] = correlations

# Prepare data for plotting
x = list(corr_distribution.keys())
y_mean = [np.mean(corr_distribution[p]) for p in x]
y_q1 = [np.percentile(corr_distribution[p], 25) for p in x]
y_q3 = [np.percentile(corr_distribution[p], 75) for p in x]

# Plot mean + IQR band
plt.figure(figsize=(8, 5))
plt.plot(x, y_mean, label="Mean Spearman Correlation", marker="o")
plt.fill_between(x, y_q1, y_q3, alpha=0.3, label="IQR (25–75%)")
plt.axhline(y=0.8, color='gray',linestyle='--')

plt.xlabel("Percentage of Vitamin D Targets Retained")
plt.ylabel("Spearman Rank Correlation")
plt.ylim(0, 1.1)
plt.xlim(0, 110)
plt.legend()
plt.tight_layout()

plt.savefig("../../sup_plots/robustness/2_3_rank_correlation_distribution.png", dpi=600, bbox_inches='tight')
plt.show()


The correlation decreases with smaller target sets. However, a Spearman correlation larger than 0.8 indicates that the rankings remain stable even when up to 20% of the targets are missing.